In [6]:

from rcpm import dataset
from rcpm import utils
from rcpm import eval

from pathlib import Path

import os

In [7]:
raw_data_folder = Path('/data')
cropped_image_dir = raw_data_folder / 'cropped_data'
split_file_dir = utils.file_abs_path(os.getcwd()) / '_test/files/wp_reid_info.json'

In [8]:
wp_reid_dataset = dataset.DataBank(
    minframes=3,
    raw_data_folder=raw_data_folder,
    cropped_image_dir=cropped_image_dir,
    split_file_dir=split_file_dir
)

Load data <--- /Users/jangsoopark/Documents/course/2023/fall/internet-of-things/WP-ReID/_test/files/wp_reid_info.json
WPReID dataset
Gallery ID diff Probe ID: []

+--------+--------+-------+---------+
|  Raw   |  Test  | Probe | Gallery |
+--------+--------+-------+---------+
|  #ID   |   79   |   79  |    79   |
| #Track |  868   |  239  |   868   |
| #Image | 106578 | 33345 |  106578 |
|  #Cam  |   6    |   6   |    6    |
| MaxLen |  1165  |  1165 |   1165  |
| MinLen |   3    |   3   |    3    |
| AvgLen |  122   |  139  |   122   |
+--------+--------+-------+---------+
After drop videos less than: test  3 frames, check cam number
No related gallery track with different camera id. Drop probe [   33     3 96929 97207   278]
No related gallery track with different camera id. Drop probe [   34     3 97207 97258    51]
No related gallery track with different camera id. Drop probe [   36     3 97342 97522   180]
No related gallery track with different camera id. Drop probe [   37     3 

In [9]:
files_dir = utils.file_abs_path(os.getcwd()) / '_test/files'

mmt_duke_dist_file = files_dir / 'mmt_duke_g2g_distmat.json'

In [10]:
import copy
from rcpm import update_module 
from rcpm import video_reid_performance
from rcpm import eval_tools
# import get_vision_record_dist, visual_affinity_update, trajectory_distance_update, norm_data

class AblationRecurrentContextPropagationModule(eval.RecurrentContextPropagationModule):

    def rcpm(self, gt_dist_new, iteration=1):
        distMat_new = update_module.visual_affinity_update(
            self.raw_distMat, 
            gt_dist=gt_dist_new.copy(), 
            T=self.delta, 
            alpha=self.alpha
        )
        cmc_reid, mAP_reid = video_reid_performance.compute_video_cmc_map(
            distMat_new[self.query_index][:, self.gallery_index], 
            self.query_id, self.gallery_id,
            self.query_cam_id, 
            self.gallery_cam_id
        )
        gt_dist_new = update_module.trajectory_distance_update(
            distMat_new, 
            self.raw_gt_dist.copy(), 
            k=self.k
        )
        cmc_SM = eval_tools.get_signal_match_cmc(
            gt_dist_new[self.query_index].copy(), 
            self.gt_dist_is_inf[self.query_index].copy(),
            self.query_id.copy(), 
            self.traj_id.copy()
        )
        
        return gt_dist_new, cmc_reid, mAP_reid, cmc_SM
        
    def __call__(self, *args, **kwargs):
        print('K={}, Delta={}, Iteration={}'.format(self.k, self.delta, self.iters))
        cmc_reid, mAP_reid = video_reid_performance.compute_video_cmc_map(
            self.raw_distMat[self.query_index][:, self.gallery_index], 
            self.query_id, self.gallery_id,
            self.query_cam_id, 
            self.gallery_cam_id
        )
        cmc_SM = eval_tools.get_signal_match_cmc(
            self.raw_gt_dist[self.query_index].copy(), 
            self.gt_dist_is_inf[self.query_index].copy(),
            self.query_id.copy(), 
            self.traj_id.copy()
        )

        # print('Iteration {}: ReID rank-1 {:.2f} mAP {:.2f}. Signal Matching rank-1 {:.2f}'.format(
        #     0, cmc_reid[0] * 100, mAP_reid * 100, cmc_SM[0] * 100))
        
        last_gt_dist = self.raw_gt_dist
        for i in range(self.iters):
            last_gt_dist, cmc_reid, mAP_reid, cmc_SM = self.rcpm(last_gt_dist, iteration=i+1)
            # print('Iteration {}: ReID rank-1 {:.2f} mAP {:.2f}. Signal Matching rank-1 {:.2f}'.format(
            #     i + 1, cmc_reid[0] * 100, mAP_reid * 100, cmc_SM[0] * 100))

        return cmc_reid, mAP_reid, cmc_SM


def ablation_update_with_gps(distMat, all_info, query_index, gallery_index, bndbox_gps, trajectory, k=9, delta=74, iters=5):
    '''
    :param distMat: (g, g), the distance between gallery tracklet with gallery tracklet
    :param all_info: (num, 5), num is the total number of gallery tracklet, each row [p_id, cam_id, idx, idx+frame_num, frame_num]
    :param query_index: the query index for query tracklet in gallery tracklet
    :param gallery_index: the gallery index for query tracklet in gallery tracklet
    :param bndbox_gps: (image_num, 3), each raw: [time_id, gps_j, gps_w]
    :param trajectory: dict, the key is the corresponding person id. trajectory[p_id] : (n, 3), each row [timestamp, gps_j, gps_w]
    :return:
    '''
    gallery_info = all_info[gallery_index]
    gt_gps_dist, traj_id = eval.get_vision_record_dist(gallery_info.copy(), bndbox_gps.copy(), copy.deepcopy(trajectory))
    rcpm = AblationRecurrentContextPropagationModule(k=k, delta=delta, iters=iters, distMat=distMat, gt_dist=gt_gps_dist, traj_id=traj_id)
    rcpm.init_dataset_info(all_info=all_info, query_index=query_index, gallery_index=gallery_index)
    
    return rcpm()

In [11]:
k = 9
sigma = 30
max_iters = 4

distmat = utils.DataPacker.load(mmt_duke_dist_file)['g2g_distmat']
cmc_reid, mAP_reid, cmc_SM = ablation_update_with_gps(
    distmat, 
    wp_reid_dataset.test_info, 
    wp_reid_dataset.probe_index, 
    wp_reid_dataset.gallery_index,
    wp_reid_dataset.gps_info, 
    wp_reid_dataset.trajectory_info, 
    k=k,
    delta=sigma,
    iters=max_iters
)

print(cmc_reid, mAP_reid, cmc_SM)

Load data <--- /Users/jangsoopark/Documents/course/2023/fall/internet-of-things/WP-ReID/_test/files/mmt_duke_g2g_distmat.json
K=9, Delta=30, Iteration=4


/Users/jangsoopark/Documents/course/2023/fall/internet-of-things/WP-ReID/src/rcpm/update_module.py:93: RuntimeWarning: invalid value encountered in multiply
  tg_dist_wighted_i = tg_dist_i * dist_weight


[0.73134327 0.75124377 0.7711443  0.7960199  0.80099505] 0.3762999577308769 [0.63414633 0.7012195  0.74390244 0.7804878  0.79268295]


In [13]:

from ray.tune.schedulers import ASHAScheduler
from ray.tune import CLIReporter
from ray import tune

os.environ["RAY_DEDUP_LOGS"] = "0"

search_space = {
    'k': tune.grid_search(list(range(1, 7))),
    'delta': tune.grid_search(list(range(60, 80))),
    'iters': tune.grid_search(list(range(1, 9))),
}


def objective(config):
    cmc_reid, mAP_reid, cmc_SM = ablation_update_with_gps(
        distmat, 
        wp_reid_dataset.test_info, 
        wp_reid_dataset.probe_index, 
        wp_reid_dataset.gallery_index,
        wp_reid_dataset.gps_info, 
        wp_reid_dataset.trajectory_info, 
        k=config['k'],
        delta=config['delta'],
        iters=config['iters']
    )
    
    return {
        'reid_r1': cmc_reid[0],
        'mAP': mAP_reid,
        'sm_r1': cmc_SM[0]
    }

tuner = tune.Tuner(
    objective, param_space=search_space,
    
    tune_config=tune.TuneConfig(
        num_samples=1
    )
)
results = tuner.fit()
print(results.get_best_result(metric="mAP", mode="max").config)


2023-12-05 01:44:20,906	WARNING worker.py:2074 -- Warning: The actor ImplicitFunc is very large (20 MiB). Check that its definition is not implicitly capturing a large array or other object in scope. Tip: use ray.put() to put large objects in the Ray object store.
(raylet) /Users/jangsoopark/Documents/course/2023/fall/internet-of-things/WP-ReID/venv/lib/python3.9/site-packages/urllib3/__init__.py:34: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
(raylet)   warnings.warn(
(raylet) /Users/jangsoopark/Documents/course/2023/fall/internet-of-things/WP-ReID/venv/lib/python3.9/site-packages/urllib3/__init__.py:34: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
(raylet)   warnings.warn(
(raylet) /Users/jangsoopark/Documents/course/2023/fall/in

(objective pid=7215) K=1, Delta=70, Iteration=1


(objective pid=7215) /Users/jangsoopark/Documents/course/2023/fall/internet-of-things/WP-ReID/src/rcpm/update_module.py:93: RuntimeWarning: invalid value encountered in multiply
(objective pid=7215)   tg_dist_wighted_i = tg_dist_i * dist_weight


(objective pid=7220) K=1, Delta=70, Iteration=2 [repeated 12x across cluster] (Ray deduplicates logs by default. Set RAY_DEDUP_LOGS=0 to disable log deduplication, or see https://docs.ray.io/en/master/ray-observability/ray-logging.html#log-deduplication for more options.)
(objective pid=7220) K=1, Delta=65, Iteration=3 [repeated 12x across cluster]
(objective pid=7213) K=1, Delta=67, Iteration=4 [repeated 12x across cluster]
(objective pid=7210) K=1, Delta=66, Iteration=5 [repeated 10x across cluster]
(objective pid=7220) K=1, Delta=69, Iteration=5 [repeated 2x across cluster]
(objective pid=7219) K=1, Delta=66, Iteration=6 [repeated 8x across cluster]
(objective pid=7220) K=1, Delta=70, Iteration=6 [repeated 4x across cluster]
(objective pid=7216) K=1, Delta=66, Iteration=7 [repeated 7x across cluster]
(objective pid=7217) K=1, Delta=70, Iteration=7 [repeated 3x across cluster]
(objective pid=7220) K=1, Delta=71, Iteration=7 [repeated 2x across cluster]


2023-12-05 01:46:46,344	WARNING tune.py:186 -- Stop signal received (e.g. via SIGINT/Ctrl+C), ending Ray Tune run. This will try to checkpoint the experiment state one last time. Press CTRL+C (or send SIGINT/SIGKILL/SIGTERM) to skip. 


(objective pid=7216) K=1, Delta=67, Iteration=8 [repeated 7x across cluster]



KeyboardInterrupt



(objective pid=7219) K=1, Delta=68, Iteration=8
